In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import math
from decimal import Decimal
import spacy

In [2]:
'''Loading the Universal Sentence Encoder'''

def embed_model(model):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(model)
        #Reduces Logging
        tf.logging.set_verbosity(tf.logging.ERROR)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

embedding_model = embed_model("embedding_model/")

Instructions for updating:
Colocations handled automatically by placer.


W0829 10:32:29.670535 10008 deprecation.py:323] From C:\Users\Barshana\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [8]:
'''Loading the input and pre-processing'''
final_df = pd.read_csv('final_pre-processed_input.csv',sep=';',index_col=0)
final_df[final_df['label']==1.0].head(100)

,accommodation_id,basename,at,description,value_type_id,amenities_id,amenities_cont,label,sorted_amenities_id,a1,a2,a3,a4,a5,a6,a7
13,7321,Tajaraste,8,Tucked away in a quieter part of Playa del Ing...,1,"[47, 255]","[0, 1]",1.0,"[0, 0, 0, 47, 0, 0, 255]",0,0,0,47,0,0,255
14,7637,Anthos Apartments,8,Anthos Apartments is conveniently located in t...,1,[255],[1],1.0,"[0, 0, 0, 0, 0, 0, 255]",0,0,0,0,0,0,255
45,15974,TownePlace Suites Orlando East/UCF Area,6,TownePlace Suites by Marriott is an extended-s...,1,"[63, 48, 255]","[0, 1, 1]",1.0,"[0, 0, 0, 0, 48, 63, 255]",0,0,0,0,48,63,255
103,24329,Relais Rosa dei Venti,8,The Relais Rosa dei Veni Residence lies in Mon...,1,[255],[1],1.0,"[0, 0, 0, 0, 0, 0, 255]",0,0,0,0,0,0,255
149,31121,Kristall,8,Appartement Kristall is conveniently located i...,1,[255],[1],1.0,"[0, 0, 0, 0, 0, 0, 255]",0,0,0,0,0,0,255
165,33394,Green Village,6,This 4 star hotel is located in the city centr...,1,"[63, 47, 13, 255]","[1, 1, 1, 1]",1.0,"[13, 0, 0, 47, 0, 63, 255]",13,0,0,47,0,63,255
169,33592,Vangelis Hotel Apartments,6,"Ideally located at the center of Protaras, Van...",1,"[48, 47, 255]","[1, 1, 0]",1.0,"[0, 0, 0, 47, 48, 0, 255]",0,0,0,47,48,0,255
196,37955,Aparthotel Nereida,6,With its spectacular sea views and nearby beac...,1,"[48, 47, 255]","[1, 1, 1]",1.0,"[0, 0, 0, 47, 48, 0, 255]",0,0,0,47,48,0,255
206,38886,Giannis & Fotini,6,The 1-star Giannis&Foteini offers comfort and ...,1,"[63, 255]","[1, 1]",1.0,"[0, 0, 0, 0, 0, 63, 255]",0,0,0,0,0,63,255
233,43034,Nautibeach Owners Rental Pool,6,Conveniently located minutes from Mundaca Hous...,1,"[47, 255]","[1, 1]",1.0,"[0, 0, 0, 47, 0, 0, 255]",0,0,0,47,0,0,255


In [9]:
print(final_df['description'][895])

A 10-minute walk from Four Mile Beach, the casual holiday accommodation, At the Mango Tree, is rated 3.5-stars and is also within walking distance of the Bally Hooley Steam Railway.
A mixture of apartments and townhouses are available. All feature free Wi-Fi access, flat-screen television with DVD player, full kitchen, sofa, and a patio or balcony. Each room also has a bathroom with a shower and hairdryer.
Featured at the At the Mango Tree is an outdoor saltwater pool surrounded by gardens. There's also a tennis court while onsite parking is free.
Guests at the At the Mango Tree can enjoy a casual meal in the barbecue area.
The hotel is only a 15-minute walk from the centre of Port Douglas and Crystalbrook Marina. Guests can also reach the famous Rainforest Habitat in about five minutes by car.


In [10]:
'''Embedding the items using the Encoder Model and calculate the similarity'''

descriptions_list = final_df['description'].tolist()[895:896]
print(descriptions_list)
# Loop through all the items from the input csv, calculate the embeddings and similarity with keywords
embedding_item_list = embedding_model(descriptions_list)
print(embedding_item_list)

["A 10-minute walk from Four Mile Beach, the casual holiday accommodation, At the Mango Tree, is rated 3.5-stars and is also within walking distance of the Bally Hooley Steam Railway.\nA mixture of apartments and townhouses are available. All feature free Wi-Fi access, flat-screen television with DVD player, full kitchen, sofa, and a patio or balcony. Each room also has a bathroom with a shower and hairdryer.\nFeatured at the At the Mango Tree is an outdoor saltwater pool surrounded by gardens. There's also a tennis court while onsite parking is free.\nGuests at the At the Mango Tree can enjoy a casual meal in the barbecue area.\nThe hotel is only a 15-minute walk from the centre of Port Douglas and Crystalbrook Marina. Guests can also reach the famous Rainforest Habitat in about five minutes by car."]
[[-8.27748179e-02 -2.25545354e-02  3.89277266e-04 -5.81105500e-02
  -1.37256719e-02  8.32810774e-02 -6.93235323e-02  2.87133809e-02
  -4.83575687e-02  1.45252598e-02  3.40293460e-02 -3.6

   5.58841266e-02 -5.88414781e-02 -3.10384594e-02  5.23529500e-02]]


In [65]:
embedding_item_list = np.array(embedding_item_list)
embedding_item_list.shape

(6000, 512)

In [66]:
reshaped_embeddings = np.array([np.reshape(embed, (len(embed), 1)) for embed in embedding_item_list])
reshaped_embeddings.shape

(6000, 512, 1)

In [67]:
labels_one_hot = []
for label in final_df['label'].tolist():
    if label == 0:
      labels_one_hot.append([1., 0.])
    else:
      labels_one_hot.append([0., 1.])
labels_one_hot = np.array(labels_one_hot)

In [68]:
import pickle

embeddings_file = "embeddings-{}.pickle".format(len(reshaped_embeddings))
labels_file = "labels-{}.pickle".format(len(final_df['label'].tolist()))

pickle.dump(reshaped_embeddings, open(embeddings_file, 'wb'))
pickle.dump(labels_one_hot, open(labels_file, 'wb'))

In [70]:
import pickle

reshaped_embeddings = pickle.load(open('embeddings-6000.pickle', 'rb'))
labels = pickle.load(open('labels-6000.pickle', 'rb'))

In [71]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(reshaped_embeddings, labels, test_size=.3)

In [72]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4200, 512, 1), (4200, 2), (1800, 512, 1), (1800, 2))

In [74]:
vector_size = 512
batch_size = 500
no_epochs = 10

In [75]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard

model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same',
                 input_shape=(vector_size, 1)))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.3)))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

tensorboard = TensorBoard(log_dir='logs/', histogram_freq=0, write_graph=True, write_images=True)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_25 (Conv1D)           (None, 512, 32)           128       
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 512, 32)           3104      
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 512, 32)           3104      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 170, 32)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 1024)              2232320   
_________________________________________________________________
dense_23 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
__________

In [76]:
model.fit(np.array(x_train), np.array(y_train), batch_size=batch_size, epochs=no_epochs,
         validation_data=(np.array(x_test), np.array(y_test)), callbacks=[tensorboard, EarlyStopping(min_delta=0.0001, patience=3)])

Train on 4200 samples, validate on 1800 samples
Epoch 1/10
4200/4200 [==============================] - ETA: 3:31 - loss: 0.8165 - acc: 0.396 - ETA: 2:44 - loss: 0.7497 - acc: 0.489 - ETA: 2:20 - loss: 0.7045 - acc: 0.546 - ETA: 1:53 - loss: 0.6612 - acc: 0.603 - ETA: 1:26 - loss: 0.6311 - acc: 0.641 - ETA: 1:00 - loss: 0.6006 - acc: 0.675 - ETA: 35s - loss: 0.5766 - acc: 0.702 - ETA: 9s - loss: 0.5596 - acc: 0.7205 - 234s 56ms/step - loss: 0.5552 - acc: 0.7262 - val_loss: 0.4403 - val_acc: 0.8489
Epoch 2/10
4200/4200 [==============================] - ETA: 3:26 - loss: 0.4460 - acc: 0.854 - ETA: 3:00 - loss: 0.4764 - acc: 0.844 - ETA: 2:33 - loss: 0.4649 - acc: 0.848 - ETA: 2:08 - loss: 0.4561 - acc: 0.852 - ETA: 1:39 - loss: 0.4540 - acc: 0.855 - ETA: 1:10 - loss: 0.4638 - acc: 0.852 - ETA: 42s - loss: 0.4699 - acc: 0.850 - ETA: 12s - loss: 0.4731 - acc: 0.84 - 279s 66ms/step - loss: 0.4717 - acc: 0.8505 - val_loss: 0.4751 - val_acc: 0.8489
Epoch 3/10
4200/4200 [=====================

In [77]:
model.metrics_names

['loss', 'acc']

In [78]:
model.evaluate(x=x_test, y=y_test, batch_size=500, verbose=1)

1800/1800 [==============================] - ETA: 15 - ETA: 9 - ETA:  - 22s 12ms/step


[0.42542971008353764, 0.8488888872994317]

In [79]:
model.save('USE-BiLSTM.model')

In [105]:
text = ["Tucked away in a quieter part of Playa del Inglés, $hotesname is a complex of cosy bungalows located at just a ten-minute walk from the beach.Spacious and relaxed bungalows lead off onto a private terrace, with living areas featuring a satellite TV. Standard amenities include a well-equipped kitchenette including cookware, crockery and a fridge.There is free Wi-Fi throughout and free private parking. Guests are welcome to relax in the gardens, and sunbathe on the multiple loungers by the two pools, one of which is reserved for children.Tajaraste features no on-site eatery, while there is a great choice in the Tajaraste town centre, at just a five-minute walk. Guests there also have access to a plentiful shopping area including, restaurants, bars, cafés, supermarkets and pharmacies.A bus stop is just a few steps away. Guests like to visit the Playa del Inglés and its renowned Dunas de Maspalomas"]
test_input = embedding_model(text)
test_input.shape
test_embeddings = np.array(test_input)
test_embeddings.shape
t_reshaped_embeddings = np.array([np.reshape(embed, (len(embed), 1)) for embed in test_embeddings])
t_reshaped_embeddings.shape

(1, 512, 1)

In [106]:
from keras.models import load_model
model = load_model('USE-BiLSTM.model')
prediction = model.predict_classes(t_reshaped_embeddings)
print(prediction)

[0]
